# 检索器：查询重构
+ 用自然语言来查询SQL

## 查询SQL
+ 步骤一：将问题转为SQL查询，模型将用户输入转换为SQL语句
+ 步骤二：执行SQL查询，执行查询。
+ 步骤三：回答问题，模型使用查询结果响应用户输入。

In [7]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

In [1]:
! pip install langchainhub langgraph

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [langgraph]/7 [langgraph]checkpoint]


### 查看数据库

In [22]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
# print(db.get_table_info())
db.run("SELECT * FROM Artists LIMIT 10")

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### 使用Hub上的提示词

In [ ]:
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

print(query_prompt_template.pretty_print())
# assert len(query_prompt_template) == 1
# query_prompt_template.messages[0].pretty_print()

================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

================================ Human Message =================================

Question: {input}
None


AssertionError: 

In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)

In [ ]:
# 使用LCEL创建一个最简单的SQL查询
from typing_extensions import Annotated
from typing_extensions import TypedDict


# 定义结构化对象
class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str


class QueryOutput(TypedDict):
    """生成SQL查询"""

    query: Annotated[str, ..., "Syntactically valid SQL query"]


def write_query(state: State) -> State:
    """生成SQL查询"""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [35]:
sql_message = write_query({"question": "一共有多少员工?"})
print(sql_message)

OutputParserException: Invalid json output: 要查询员工的总数，我可以使用以下SQL查询：

```sql
SELECT COUNT(*) AS TotalEmployees 
FROM employees;
```

这个查询会计算employees表中的记录数量，也就是员工的总数。
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

### 得到SQL语句可以接着进行执行

In [36]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State) -> State:
    """执行SQL查询"""
    query = state["query"]
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(query)}

In [37]:
execute_query(sql_message)

NameError: name 'sql_message' is not defined

In [38]:
# 将上面内容创建一个完整的链
from langchain_core.runnables import RunnablePassthrough


# Define the chain to answer questions from SQL query
def answer_question(state: State):
    """Format answer based on the query result."""
    prompt = f"""Based on the SQL query:
    {state["query"]}
    
    And the query result:
    {state["result"]}
    
    Answer the user's question: {state["question"]}
    Provide a concise and informative response.
    """

    return {"answer": llm.invoke(prompt).contet}


# Create a full chain from question to answer
sql_chain = (
    RunnablePassthrough.assign(query=write_query)
    .assign(result=execute_query)
    .assign(answer=answer_question)
)

# 使用示例
question = "获取销售额最高的5位员工及其销售总额"
response = sql_chain.invoke({"question": question})

print("Question", question)
print("\nGenerated SQL:")
print(response["query"])
print("\nExecution Result:")
print(response["result"])
print("\nAnswer:")
print(response["answer"])

OutputParserException: Invalid json output: Here's the SQL query to get the top 5 employees with the highest sales totals:

```sql
SELECT 
    e.EmployeeId, 
    e.FirstName, 
    e.LastName, 
    SUM(i.Total) AS TotalSales
FROM 
    employees e
JOIN 
    customers c ON e.EmployeeId = c.SupportRepId
JOIN 
    invoices i ON c.CustomerId = i.CustomerId
GROUP BY 
    e.EmployeeId, e.FirstName, e.LastName
ORDER BY 
    TotalSales DESC
LIMIT 5;
```

This query:
1. Joins employees with customers they support (via SupportRepId)
2. Joins those customers with their invoices
3. Groups by employee and calculates the sum of invoice totals
4. Orders by the total sales in descending order
5. Limits to the top 5 results
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 